In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold

Первым делом загрузим основную обучающую выборку и переведём время в правильный формат

In [11]:
df = pd.read_csv("impressions.сsv")
df['event_datetime_m'] = pd.to_datetime(df.event_datetime_m, yearfirst = True)
df['dayofweek'] = [d.isoweekday() for d in df.event_datetime_m]
df['hour'] = [d.hour for d in df.event_datetime_m]
df['month'] = [d.month for d in df.event_datetime_m]
df['day'] = [d.day for d in df.event_datetime_m]
df['daymonth'] = 31*df.month + df.day
df.head()

,rank,event_datetime_m,is_clicked,id_show,id_user,dayofweek,hour,month,day,daymonth
0,1,2017-01-12 12:30:00,0.0,5547,906335,4,12,1,12,43
1,3,2017-01-12 14:53:00,0.0,5547,5038628,4,14,1,12,43
2,4,2017-01-12 17:10:00,0.0,8148,5038628,4,17,1,12,43
3,2,2017-01-12 14:48:00,0.0,8131,5038628,4,14,1,12,43
4,1,2017-01-12 14:58:00,0.0,8367,5038628,4,14,1,12,43


Затем избавимся от ненужных нам признаков, разобьём данные на тренировочную и валидационную части
</br>У нас остались категориальные признаки, плюс daymonth, теперь время аггрегировать различные признаки для id_show и id_user

In [12]:
df = df.drop(['event_datetime_m', 'month', 'day'], axis = 1)
train_df = df[df.daymonth < 117]
test_df = df[df.daymonth > 116]
#y_train = train_df.is_clicked
#train_df = train_df.drop(['is_clicked'], axis = 1)
#y_valid = test_df.is_clicked
#valid_df = test_df.drop(['is_clicked'], axis = 1)
X_train.head()

,rank,id_show,id_user,dayofweek,hour,daymonth
0,1,5547,906335,4,12,43
1,3,5547,5038628,4,14,43
2,4,8148,5038628,4,17,43
3,2,8131,5038628,4,14,43
4,1,8367,5038628,4,14,43


Теперь загрузим show_df и будем считать разлиные показатели

In [6]:
show_df = pd.read_csv("show_data.сsv", index_col = 'id_show')
show_df.head()

,organizer_id,age_category,duration,parent_genre_id,child_genre_id,IdBuilding,show_maxprice,show_minprice,show_meanprice,show_stdprice
id_show,,,,,,,,,,
0,1854307.0,4.0,10200.0,[13654],[13655],10.0,6000.0,160.0,1495.231712,777.042344
1,1854309.0,2.0,10800.0,[13654],[13658 13659],12.0,2600.0,150.0,995.630174,676.837725
2,1854327.0,5.0,8700.0,[13654],[13655],33.0,10000.0,50.0,743.746006,1153.720468
3,1854327.0,2.0,6600.0,[13679],[1868812],33.0,3000.0,50.0,707.560152,605.466747
4,1854319.0,7.0,6600.0,[13654],[13657],22.0,450.0,300.0,384.545227,62.711090


In [48]:

#[train_df.groupby(['id_show'])['is_clicked'].mean()[i] if i in train_df.groupby(['id_show'])['is_clicked'].count().index
#                            else 0 for i in range(show_df.shape[0])]
train_df = train_df.assign(mean_click = pd.Series(train_df.groupby(['id_show'])['is_clicked'].transform('mean')))
#df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
#train_df.loc[:,'click_count'] = train_df.groupby(['id_show'])['is_clicked'].transform('count')
train_df.head()

,rank,is_clicked,id_show,id_user,dayofweek,hour,daymonth,mean_clicked,click_count,mean_click
0,1,0.0,5547,906335,4,12,43,0.028986,414.0,0.028986
1,3,0.0,5547,5038628,4,14,43,0.028986,414.0,0.028986
2,4,0.0,8148,5038628,4,17,43,0.158344,12176.0,0.158344
3,2,0.0,8131,5038628,4,14,43,0.296337,20011.0,0.296337
4,1,0.0,8367,5038628,4,14,43,0.021130,20114.0,0.021130
